In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [3]:
imgs = torch.stack([img_t for img_t, _ in train_data], dim=3)

In [4]:
imgs.shape

torch.Size([1, 28, 28, 60000])

In [5]:
imgs.view(1, -1).mean(dim=1)

tensor([0.1307])

In [6]:
imgs.view(1, -1).std(dim=1)

tensor([0.3081])

In [7]:
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                   ]))
test_data = datasets.MNIST(root='data', train=False,
                                   download=True, transform=transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                    ]))

In [8]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100)

In [10]:
iter(train_loader).next()[0].size()

torch.Size([100, 1, 28, 28])

In [14]:
# definir la arquitectura
class ConvolutionalNN(nn.Module):
    def __init__(self):
        super(ConvolutionalNN, self).__init__()
        ## convolutional layers ##
        self.t_conv_1 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1)
        self.t_conv_2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
        self.max_pool_1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.max_pool_2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.linear_1 = nn.Linear(64 * 7 * 7 * 100,2500)
        self.linear_2 = nn.Linear(2500,10)

    def forward(self, x):
        ## NeuralNet ##
        result = self.t_conv_1(x)
        result = F.relu(result)
        result = self.max_pool_1(result)
        result = self.t_conv_2(result)
        result = F.relu(result)
        result = self.max_pool_2(result)
        print(result.shape)
        result = result.reshape(1, -1)
        print(result.shape)
        result = self.linear_1(result)
        result = self.linear_2(result)
        return result
model = ConvolutionalNN()

In [12]:
test_image = iter(train_loader).next()[0][0].unsqueeze(0)
test_label = iter(train_loader).next()[1][0]
print(model(test_image))
print(test_label)

torch.Size([1, 64, 7, 7])
torch.Size([1, 3136])
tensor([[ 0.0837,  0.0275,  0.0261,  0.1717,  0.1129, -0.1777,  0.0134, -0.0811,
         -0.0167, -0.0824]], grad_fn=<AddmmBackward>)
tensor(5)


In [15]:
n_epochs = 20
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        print(outputs.size())
        print(labels.size())
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        if (i + 1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

torch.Size([100, 64, 7, 7])
torch.Size([1, 313600])
torch.Size([1, 10])
torch.Size([100])


ValueError: Expected input batch_size (1) to match target batch_size (100).

In [ ]:
model.eval() with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy: {} %'.format((correct / total) * 100))